In [1]:
import os

In [2]:
%pwd

'/mnt/d/Python/MLProjectsPW/Thyroid-Disease-Prediction/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/mnt/d/Python/MLProjectsPW/Thyroid-Disease-Prediction'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tejas05in"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "9efcb5c7b79d0e949378459b922b1462a80fa413"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml , create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """
        This function returns the ModelEvaluationConfig object that contains all the configuration parameters required for model evaluation.

        Args:
            None

        Returns:
            ModelEvaluationConfig: The ModelEvaluationConfig object containing all the configuration parameters

        """
        config = self.config.model_evaluation
        params = self.params.GradientBoostedTreesLearner
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from urllib.parse import urlparse
import mlflow
import numpy as np
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from ThyroidProject.utils.common import save_json
from mlflow.data.pandas_dataset import PandasDataset

2024-02-03 00:17:04.325212: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-03 00:17:04.597260: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-03 00:17:05.795632: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 00:17:05.797239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 00:17:06.092511: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    

    def log_into_mlflow(self):
        """
        This function logs the metrics and model into mlflow.

        Args:
            None

        Returns:
            None

        """

        # Load the test data
        test_data = pd.read_csv(self.config.test_data_path)

        # Convert the pandas dataframe to a tf.data.Dataset
        test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
            test_data, label=self.config.target_column)

        # Load the model
        model = tf.keras.models.load_model(self.config.model_path)

        # Compile the model
        model.compile(metrics=["accuracy"])

        # Evaluate the model on the test data
        evaluation = model.evaluate(test_ds, return_dict=True)
        
        # Save the metrics as a local file
        # scores = {"accuray": evaluation["accuracy"], "loss": evaluation["loss"]}
        save_json(path=Path(self.config.metric_file_name), data=evaluation)

        # Set the mlflow tracking uri
        mlflow.set_registry_uri(self.config.mlflow_uri)

        # Get the type of the tracking uri
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        dataset: PandasDataset = mlflow.data.from_pandas(
            test_data)
    
        # # Start a mlflow run
        with mlflow.start_run():
            
            #log data
            mlflow.log_input(dataset, context="testing")

            # Log the parameters
            mlflow.log_params(self.config.all_params)

            # Log the metrics
            mlflow.log_metric("loss", evaluation["loss"])
            mlflow.log_metric("accuray", evaluation["accuracy"])
            
            # Register the model
            # There are other ways to use the model registry , which depends on the use case
            # Please refer to the documentation for more information
            # https://mlflow.org/docs/latest/model-registry.html #api-workflow
            if tracking_uri_type_store != 'file':
                mlflow.tensorflow.log_model(
                    model, 'model', registered_model_name="GradientBoostedTreesModel", pip_requirements ='requirements.txt')
            else:
                mlflow.tensorflow.log_model(model, 'model')

In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-02-03 00:25:24,370:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-02-03 00:25:24,379:INFO:common:yaml file: params.yaml loaded successfully]
[2024-02-03 00:25:24,385:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-02-03 00:25:24,389:INFO:common:created directory at :artifacts]
[2024-02-03 00:25:24,395:INFO:common:created directory at :artifacts/model_evaluation]


[INFO 24-02-03 00:25:24.6950 IST kernel.cc:1233] Loading model from path artifacts/model_trainer/model/assets/ with prefix 5289d509f4b547ac
[INFO 24-02-03 00:25:25.7214 IST decision_forest.cc:660] Model loaded with 111 root(s), 4117 node(s), and 14 input feature(s).
[INFO 24-02-03 00:25:25.7214 IST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-02-03 00:25:25.7214 IST kernel.cc:1061] Use fast generic engine


2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.9865
[2024-02-03 00:25:25,955:INFO:common:json file saved at:artifacts/model_evaluation/metrics.json]


/mnt/d/Python/MLProjectsPW/Thyroid-Disease-Prediction/env/lib/python3.11/site-packages/mlflow/data/digest_utils.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/mnt/d/Python/MLProjectsPW/Thyroid-Disease-Prediction/env/lib/python3.11/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#han

[2024-02-03 00:31:16,364:WARNING:connectionpool:Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/log-inputs]


2024/02/03 00:31:18 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-02-03 00:31:18,948:INFO:builder_impl:Assets written to: /tmp/tmp0hkcec80/model/data/model/assets]


/mnt/d/Python/MLProjectsPW/Thyroid-Disease-Prediction/env/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'GradientBoostedTreesModel' already exists. Creating a new version of this model...
2024/02/03 00:31:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GradientBoostedTreesModel, version 9
Created version '9' of model 'GradientBoostedTreesModel'.
